# Web scraping de literales con Python

In [11]:
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import selenium.common.exceptions as exc
import os

import time

import numpy as np
import pandas as pd

import requests
import xmltodict

from openpyxl import load_workbook

In [2]:
# Instantiate options
opts = Options()

# opts.add_argument(" — headless") # Uncomment if the headless version needed
opts.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set the location of the webdriver
chrome_driver = os.getcwd() + "\chromedriver.exe"



In [44]:
# Find list of links
url = "https://www.leroymerlin.es/sitemap-productos.xml"
response = requests.get(url)
data = xmltodict.parse(response.content)
links = [x['loc'] for x in data['urlset']['url']]
# Filter links that doesn't contain 'como-'
links = [link for link in links if not ('/como-' in link)]
# Filter links ending in 'producto' (these are empty websites)
links = [link for link in links if link[-9:] != '/producto']
links

['https://www.leroymerlin.es/muebles-armarios-ordenacion/accesorios-organizar-ropa/organizadores-ropa',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/accesorios-organizar-ropa/pantaloneros-y-portacorbatas',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/baules-cajas-y-cestas/cajas-madera',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/baules-cajas-y-cestas/cajas-apilables-y-gavetas',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/baules-cajas-y-cestas/cestas',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/baules-cajas-y-cestas/cajas-carton',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/baules-cajas-y-cestas/baules',
 'https://www.leroymerlin.es/muebles-armarios-ordenacion/cajoneras',
 'https://www.leroymerlin.es/fontaneria/tuberias-de-agua-y-griferia-de-paso/abrazaderas-metalicas-tubos',
 'https://www.leroymerlin.es/fontaneria/tuberias-de-agua-y-griferia-de-paso',
 'https://www.leroymerlin.es/fontaneria/produccion-agua-ca

In [45]:
len(links)

2210

In [23]:
def get_names(web):

    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    # Load the HTML page
    driver.maximize_window()
    driver.get(web)
    time.sleep(2)

    # Click en Aceptar cookies
    driver.find_element_by_xpath('//div[@class="cookie-banner"]//button[@id="acceptCookies"]').click()

    # Leer nombre
    name = driver.find_element_by_xpath('//h1[@id="pageTitle"]').text

    # Leer datos
    items = []
    soup = BeautifulSoup(driver.page_source)
    items += soup.find_all('h3',{'itemprop':'name model'})

    # Click en siguiente (si existe)
    nxt = driver.find_elements_by_xpath('//li[@class="next"]/a')
    if nxt:
        nxt[0].click()
    while nxt:
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        items += soup.find_all('h3',{'itemprop':'name model'})
        while True:
            try:
                nxt = driver.find_elements_by_xpath('//li[@class="next"]/a')
                if nxt:
                    nxt[0].click()
                break
            except (exc.StaleElementReferenceException, exc.ElementNotInteractableException):
                pass
    driver.close()
    return name, list(map(lambda x: x.getText(), items))

    

web = 'https://www.leroymerlin.es/pintura/pintura-paredes-techos/pintura-acrilica'
literales = get_names(web)

KeyboardInterrupt: 

In [20]:
len(literales[1])

359

In [ ]:
# Scrape all links
def scrape_all(links):
    data = pd.DataFrame(columns = ['Sección','Subsección','Literal'])
    for i,link in enumerate(links):
        print('Enlace ' + str(i) + ' de ' + str(len(links)) + ': ' + link )
        print('Hay '+ str(len(data)) + ' elementos.')
        try:
            cat = re.findall(r'https://www.leroymerlin.es/([\w-]+)', link)[0]
            name, literales = get_names(link)
            link_data = pd.DataFrame({'Sección':cat, 'Subsección':name,'Literal':literales})
            data = data.append(link_data)
        except:
            print('Fallo')
    return data
data = scrape_all(links[100:500])

Enlace 0 de 400: https://www.leroymerlin.es/fontaneria/evacuacion-de-agua/colas-y-pegamentos-para-pvc
Hay 0 elementos.
Enlace 1 de 400: https://www.leroymerlin.es/fontaneria/accesorios-fontaneria/latiguillos
Hay 10 elementos.
Enlace 2 de 400: https://www.leroymerlin.es/fontaneria/accesorios-fontaneria/aireadores
Hay 90 elementos.
Enlace 3 de 400: https://www.leroymerlin.es/fontaneria/accesorios-fontaneria/mantenimiento-reparacion
Hay 151 elementos.
Enlace 4 de 400: https://www.leroymerlin.es/fontaneria/depuradoras
Hay 207 elementos.
Enlace 5 de 400: https://www.leroymerlin.es/fontaneria/transporte-gratis-fontaneria
Hay 214 elementos.
Enlace 6 de 400: https://www.leroymerlin.es/fontaneria/extractores-bano
Hay 311 elementos.
Enlace 7 de 400: https://www.leroymerlin.es/fontaneria/brita/brita
Hay 375 elementos.
Enlace 8 de 400: https://www.leroymerlin.es/fontaneria/brita
Hay 385 elementos.
Enlace 9 de 400: https://www.leroymerlin.es/fontaneria/ofertas-fontaneria
Hay 385 elementos.
Enlace 1

Enlace 69 de 400: https://www.leroymerlin.es/construccion/evacuacion-agua/arquetas-tapas-registro
Hay 2899 elementos.
Enlace 70 de 400: https://www.leroymerlin.es/construccion/evacuacion-agua/canaletas-evacuacion-agua
Hay 2923 elementos.
Enlace 71 de 400: https://www.leroymerlin.es/construccion/evacuacion-agua/armarios-contadores
Hay 2994 elementos.
Enlace 72 de 400: https://www.leroymerlin.es/construccion/evacuacion-agua/tubos-pvc
Hay 2996 elementos.
Enlace 73 de 400: https://www.leroymerlin.es/construccion/venta-telefonica-placas-carton-yeso
Hay 3236 elementos.
Enlace 74 de 400: https://www.leroymerlin.es/construccion/carretillas-rampas
Hay 3262 elementos.
Enlace 75 de 400: https://www.leroymerlin.es/construccion/maderas-para-la-construccion
Hay 3278 elementos.
Enlace 76 de 400: https://www.leroymerlin.es/construccion/venta-telefonica-morteros-cementos
Hay 3376 elementos.
Enlace 77 de 400: https://www.leroymerlin.es/construccion/bloques-vidrio
Hay 3410 elementos.
Enlace 78 de 400: ht

Enlace 138 de 400: https://www.leroymerlin.es/construccion/productos-para-reparaciones-de-cubiertas-suelos-y-terrazas
Hay 5241 elementos.
Enlace 139 de 400: https://www.leroymerlin.es/construccion/ofertas-aniversario-construccion
Hay 5321 elementos.
Enlace 140 de 400: https://www.leroymerlin.es/construccion/elevadores-placa
Hay 5328 elementos.
Enlace 141 de 400: https://www.leroymerlin.es/construccion/decoracion-de-terrazas--jardin-y-patios/jardineras-de-hormigon
Hay 5336 elementos.
Enlace 142 de 400: https://www.leroymerlin.es/construccion/decoracion-de-terrazas--jardin-y-patios/paso-japones
Hay 5348 elementos.
Enlace 143 de 400: https://www.leroymerlin.es/construccion/decoracion-de-terrazas--jardin-y-patios/cantos-rodados
Hay 5361 elementos.
Enlace 144 de 400: https://www.leroymerlin.es/construccion/decoracion-de-terrazas--jardin-y-patios/estabilizadores-de-grava
Hay 5414 elementos.
Enlace 145 de 400: https://www.leroymerlin.es/construccion/decoracion-de-terrazas--jardin-y-patios/acc

Enlace 200 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/lavado-y-planchado/tablas-de-planchar
Hay 11667 elementos.
Enlace 201 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/lavado-y-planchado/fundas-tablas-de-planchar
Hay 11676 elementos.
Enlace 202 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/lavado-y-planchado/cuerda-para-tender
Hay 11683 elementos.
Enlace 203 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/armarios-y-taquillas/armarios-metalicos-y-taquillas
Hay 11700 elementos.
Enlace 204 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/armarios-y-taquillas/taquillas-metalicas-de-vestuario
Hay 11723 elementos.
Enlace 205 de 400: https://www.leroymerlin.es/ferreteria-y-seguridad/armarios-y-taquillas/armarios-escoberos-y-multiusos
Hay 12083 elementos.
Enlace 206 de 400: https://www.leroymerlin.es/electricidad/clavijas-ladrones-casquillos/accesorios-para-lamparas
Hay 12114 elementos.
Enlace 207 de 400: https://www.leroymerlin.es/e

Fallo
Enlace 268 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/fontini-bf-18
Hay 20089 elementos.
Enlace 269 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/serie-garby
Hay 20121 elementos.
Enlace 270 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/fontini-kristal
Hay 20157 elementos.
Enlace 271 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/simon-270
Hay 20175 elementos.
Enlace 272 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/new-unica
Hay 20253 elementos.
Enlace 273 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/packs-de-mecanismos-con-marco
Hay 20503 elementos.
Enlace 274 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/mureva
Hay 20583 elementos.
Enlace 275 de 400: https://www.leroymerlin.es/electricidad/interruptores-y-enchufes/simon-27
Hay 20605 elementos.
Enlace 276 de 400: https://www.leroymerlin.es/

In [ ]:
len(data)

In [ ]:
data.to_excel('literales_leroymerlin100-499.xlsx')